In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llama-3.1/transformers/8b-instruct/1/model.safetensors.index.json
/kaggle/input/llama-3.1/transformers/8b-instruct/1/model-00003-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/1/config.json
/kaggle/input/llama-3.1/transformers/8b-instruct/1/model-00001-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/1/README.md
/kaggle/input/llama-3.1/transformers/8b-instruct/1/USE_POLICY.md
/kaggle/input/llama-3.1/transformers/8b-instruct/1/tokenizer.json
/kaggle/input/llama-3.1/transformers/8b-instruct/1/tokenizer_config.json
/kaggle/input/llama-3.1/transformers/8b-instruct/1/model-00004-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/1/special_tokens_map.json
/kaggle/input/llama-3.1/transformers/8b-instruct/1/model-00002-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/1/generation_config.json
/kaggle/input/banglamcq-dataset/BanglaMCQ-Dataset.csv


In [ ]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U accelerate
%pip install -U peft
%pip install -U trl

In [ ]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from trl import setup_chat_format
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          pipeline, 
                          logging)
from sklearn.metrics import (accuracy_score, 
                             classification_report, 
                             confusion_matrix)
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_csv("/kaggle/input/banglamcq-dataset/BanglaMCQ-Dataset.csv")
df.head()

,context,question,ans_txt,answer0,answer1,answer2,answer3,label
0,বাংলাদেশ একটি ক্ষুদ্র আয়তনের জনবহুল দেশ এ দেশে...,বাংলাদেশের আয়তন কত বর্গকিলোমিটার,১৪৭৫৭০,১৪০৫৭০,১৪৭৫৭০,৪৭৫৭০,৫৬৭৫৭০,1.0
1,বাংলাদেশ একটি ক্ষুদ্র আয়তনের জনবহুল দেশ এ দেশে...,মোট আয়তনের কত শতাংশ বনভূমি,১৭,২০,১৭,৫৬,৩০,1.0
2,বাংলাদেশ একটি ক্ষুদ্র আয়তনের জনবহুল দেশ এ দেশে...,বাংলাদেশের রাজধানীর নাম কি,ঢাকা,ঢাকা,চট্টগ্রাম,সিলেট,বরিশাল,0.0
3,বাংলাদেশ একটি ক্ষুদ্র আয়তনের জনবহুল দেশ এ দেশে...,বাংলাদেশে মোট কয়টি বিভাগীয় শহর রয়েছে,আটটি,দশটি,বিশটি,আটটি,এগারোটি,2.0
4,বাংলাদেশের মোট জনসংখ্যা প্রায় ১৬ কোটির মতো পৃথ...,বাংলাদেশের মোট জনসংখ্যা কয় কোটি,১৬,১৫,১৮,৫৬,১৬,3.0


In [5]:
# Shuffle the DataFrame and select only 3000 rows (adjust as needed)
df = df.sample(frac=1, random_state=85).reset_index(drop=True)[:1500]

In [6]:
# Handle unexpected label values
def clean_label(label):
    if pd.isna(label) or label not in [0, 1, 2, 3]:
        return np.nan
    return int(label)

df['label'] = df['label'].apply(clean_label)
df = df.dropna(subset=['label'])

In [7]:
df['label'] = df['label'].astype(int)

In [8]:
# Split the DataFrame
train_size = 0.8
eval_size = 0.1
train_end = int(train_size * len(df))
eval_end = train_end + int(eval_size * len(df))
X_train = df[:train_end]
X_eval = df[train_end:eval_end]
X_test = df[eval_end:]

In [ ]:
# Define prompt generation functions
def generate_prompt(data_point):
    pass # needed to create prompt funtion

In [10]:
# Generate prompts for training and evaluation data
X_train.loc[:,'text'] = X_train.apply(generate_prompt, axis=1)
X_eval.loc[:,'text'] = X_eval.apply(generate_prompt, axis=1)

# Convert to datasets
train_data = Dataset.from_pandas(X_train[["text"]])
eval_data = Dataset.from_pandas(X_eval[["text"]])

# Generate test prompts and extract true labels
y_true = X_test.loc[:,'label']
X_test = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["text"])

/tmp/ipykernel_36/3965882420.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.loc[:,'text'] = X_train.apply(generate_prompt, axis=1)
/tmp/ipykernel_36/3965882420.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_eval.loc[:,'text'] = X_eval.apply(generate_prompt, axis=1)


In [ ]:
# Prediction function
def predict(test, model, tokenizer):
    # y_pred = []
    # options = ['ক', 'খ', 'গ', 'ঘ']
    
    # for i in tqdm(range(len(test))):
    #     prompt = test.iloc[i]["text"]
    #     pipe = pipeline(task="text-generation", 
    #                     model=model, 
    #                     tokenizer=tokenizer, 
    #                     max_new_tokens=2, 
    #                     temperature=0.1)
        
    #     result = pipe(prompt)
    #     answer = result[0]['generated_text'].split("সঠিক উত্তর:")[-1].strip()
        
    #     for idx, option in enumerate(options):
    #         if option in answer:
    #             y_pred.append(idx)
    #             break
    #     else:
    #         y_pred.append(-1)  # Invalid prediction
    
    # return y_pred

In [13]:
# Evaluation function
def evaluate(y_true, y_pred):
    labels = [0, 1, 2, 3]
    
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')
    
    for label in labels:
        label_indices = [i for i in range(len(y_true)) if y_true[i] == label]
        label_y_true = [y_true[i] for i in label_indices]
        label_y_pred = [y_pred[i] for i in label_indices]
        label_accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for option {["ক", "খ", "গ", "ঘ"][label]}: {label_accuracy:.3f}')
        
    class_report = classification_report(y_true=y_true, y_pred=y_pred, target_names=["ক", "খ", "গ", "ঘ"], labels=labels)
    print('\nClassification Report:')
    print(class_report)
    
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=labels)
    print('\nConfusion Matrix:')
    print(conf_matrix)

In [11]:
# Model setup
base_model_name = "/kaggle/input/llama-3.1/transformers/8b-instruct/1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
)

model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    torch_dtype="float16",
    quantization_config=bnb_config, 
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(base_model_name)
tokenizer.pad_token_id = tokenizer.eos_token_id

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [14]:
# y_pred_org = predict(X_test, model, tokenizer)

100%|██████████| 149/149 [04:36<00:00,  1.86s/it]


In [15]:
# y_ture_list = y_true.tolist()

In [17]:
# Compare with the original model
print("\nOriginal Model Evaluation:")
evaluate(y_ture_list, y_pred_org)

Accuracy: 0.799
Accuracy for option ক: 0.684
Accuracy for option খ: 0.882
Accuracy for option গ: 0.923
Accuracy for option ঘ: 0.711

Classification Report:
              precision    recall  f1-score   support

           ক       0.96      0.68      0.80        38
           খ       0.59      0.88      0.71        34
           গ       0.84      0.92      0.88        39
           ঘ       0.96      0.71      0.82        38

    accuracy                           0.80       149
   macro avg       0.84      0.80      0.80       149
weighted avg       0.84      0.80      0.80       149


Confusion Matrix:
[[26 11  1  0]
 [ 0 30  3  1]
 [ 0  3 36  0]
 [ 1  7  3 27]]


In [14]:
# Find all linear module names
def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

modules = find_all_linear_names(model)

In [15]:
# Define LoRA configuration
lora_config = LoraConfig(
    r=4,
    lora_alpha=8,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=modules
)

# Set up training arguments
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    save_steps=500,
    logging_steps=500,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="none"
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=lora_config,
    dataset_text_field="text",
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/1188 [00:00<?, ? examples/s]

Map:   0%|          | 0/148 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [16]:
# Train the model
trainer.train()

Step,Training Loss
500,0.235400
1000,0.061600
1500,0.045500


TrainOutput(global_step=1782, training_loss=0.1023531035006916, metrics={'train_runtime': 11011.1385, 'train_samples_per_second': 0.324, 'train_steps_per_second': 0.162, 'total_flos': 8.132727747966566e+16, 'train_loss': 0.1023531035006916, 'epoch': 3.0})

In [17]:
# Save the trained model
trainer.model.save_pretrained("./fine_tuned_model")

In [23]:
# Load the fine-tuned model
fine_tuned_model = AutoModelForCausalLM.from_pretrained(
    "/kaggle/working/fine_tuned_model",
    device_map="auto",
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
)

ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [26]:
# Make predictions using the fine-tuned model
y_pred_fine_tuned = predict(X_test, model, tokenizer)

100%|██████████| 149/149 [04:54<00:00,  1.98s/it]


In [30]:
y_ture_list = y_true.tolist()

In [32]:
# Evaluate the fine-tuned model
print("Fine-tuned Model Evaluation:")
evaluate(y_ture_list, y_pred_fine_tuned)

Fine-tuned Model Evaluation:
Accuracy: 0.926
Accuracy for option ক: 0.842
Accuracy for option খ: 1.000
Accuracy for option গ: 0.923
Accuracy for option ঘ: 0.947

Classification Report:
              precision    recall  f1-score   support

           ক       0.97      0.84      0.90        38
           খ       0.79      1.00      0.88        34
           গ       0.97      0.92      0.95        39
           ঘ       1.00      0.95      0.97        38

    accuracy                           0.93       149
   macro avg       0.93      0.93      0.93       149
weighted avg       0.94      0.93      0.93       149


Confusion Matrix:
[[32  5  1  0]
 [ 0 34  0  0]
 [ 0  3 36  0]
 [ 1  1  0 36]]
